# Plastics production & recycling

In [1]:
import pickle
with open("plastics_sankey.pickle", "rb") as f:
    saved = pickle.load(f)

In [2]:
sankey_eqns = saved["sankey"]
data = saved["data"]

In [3]:
import sympy as sy
Z_product = sy.IndexedBase("Z_product")
Z_eol = sy.IndexedBase("Z_eol")
RR = sy.IndexedBase("RR")  # recycling rate
a_PE_PP_mech_recyclable = sy.Symbol("a_PE_PP_mech_recyclable")

from sankey_definitions import polymer_objects

In [4]:
from attrs import evolve
def subs_in_sankey_data(data, values):
    new_links = [
        evolve(link, data={k: v.xreplace({**values}) for k, v in link.data.items()})
        for link in data.links
    ]
    new_data = evolve(data, links=new_links)
    return new_data

In [5]:
w = subs_in_sankey_data(sankey_eqns, data).to_widget(width=1400, height=600)
w.scale = 15
w

SankeyWidget(groups=[{'id': '__wpt1_ChemicalRecycling_0', 'type': 'group', 'title': '', 'nodes': ['__wpt1_Chem…

In [6]:
from ipywidgets import interact, FloatSlider
style = {'description_width': 'initial'}
@interact(d1=FloatSlider(1.0, min=0., max=3., description="Demand 1", style=style),
          rr=FloatSlider(0.3, min=0., max=1., description="Recycling rate", style=style),
          a=FloatSlider(0.5, min=0., max=1., description="Mech recycling", style=style))
def calc_flows(d1, rr, a):
    d = {**data, Z_product[1]: d1, a_PE_PP_mech_recyclable: a}
    for k in range(len(polymer_objects)):
        d[RR[k]] = rr
    new_w = subs_in_sankey_data(sankey_eqns, d).to_widget()
    w.links = new_w.links

interactive(children=(FloatSlider(value=1.0, description='Demand 1', max=3.0, style=SliderStyle(description_wi…